<a href="https://colab.research.google.com/github/timbocf/image-to-video-generator/blob/main/ComfyUI_colab_i2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

===== Colab bootstrap for fast ComfyUI startup (GPU-ready + updates + Drive caching) =====

In [ ]:
# One cell: mounts Drive, caches pip, clones/updates repos, links models/nodes, installs torch, starts ComfyUI and (optionally) exposes it via ngrok.
# Adapt paths if you prefer a different Drive folder.
# 1) Mount Google Drive

from google.colab import drive drive.mount('/content/drive', force_remount=True)

import os, subprocess, shutil, sys, time, json, pathlib

DRIVE_ROOT = '/content/drive/MyDrive/ComfyUI' os.makedirs(DRIVE_ROOT, exist_ok=True) MODELS_DRIVE = os.path.join(DRIVE_ROOT, 'models') NODES_DRIVE = os.path.join(DRIVE_ROOT, 'custom_nodes') CACHE_DRIVE = os.path.join(DRIVE_ROOT, 'pipcache') NGROK_TOKEN_FILE = os.path.join(DRIVE_ROOT, 'ngrok_token.txt')

os.makedirs(MODELS_DRIVE, exist_ok=True) os.makedirs(NODES_DRIVE, exist_ok=True) os.makedirs(CACHE_DRIVE, exist_ok=True)

# 2) Set pip cache to Drive for fast repeated installs
os.environ['PIP_CACHE_DIR'] = CACHE_DRIVE

# 3) Clone or update ComfyUI repo
REPO_DIR = '/content/ComfyUI' COMFYUI_GIT = 'https://github.com/comfyanonymous/ComfyUI.git' # common canonical repo (modify if you use a fork) if not os.path.exists(REPO_DIR): subprocess.run(['git', 'clone', '--depth', '1', COMFYUI_GIT, REPO_DIR], check=True) else: # try to pull new changes try: subprocess.run(['git', '-C', REPO_DIR, 'fetch', '--depth', '1'], check=True) subprocess.run(['git', '-C', REPO_DIR, 'reset', '--hard', 'origin/master'], check=True) except Exception as e: print("Warning: git update failed:", e)

# 4) Symlink models and custom nodes into ComfyUI so they persist and avoid re-downloads
REPO_MODELS = os.path.join(REPO_DIR, 'models') REPO_CUSTOM_NODES = os.path.join(REPO_DIR, 'custom_nodes')

def safe_link(src, dst): if os.path.islink(dst) or os.path.exists(dst): try: if os.path.islink(dst): os.unlink(dst) elif os.path.isdir(dst): shutil.rmtree(dst) else: os.remove(dst) except Exception as e: print("Warning removing existing:", dst, e) os.symlink(src, dst)

safe_link(MODELS_DRIVE, REPO_MODELS) safe_link(NODES_DRIVE, REPO_CUSTOM_NODES)

# 5) Create a lightweight venv to keep things clean (Colab already isolated, but venv helps)
VENV_DIR = '/content/venv_comfy' if not os.path.exists(VENV_DIR): subprocess.run([sys.executable, '-m', 'venv', VENV_DIR], check=True) bin_py = os.path.join(VENV_DIR, 'bin', 'python') pip_cmd = os.path.join(VENV_DIR, 'bin', 'pip')

# Ensure pip is updated
subprocess.run([pip_cmd, 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True)

# 6) Install PyTorch appropriate for runtime (attempt CUDA install if GPU available)
# Try to detect CUDA version available on Colab: usually CUDA 11.8 or 12.x on modern runtimes.
gpu_info = !nvidia-smi -L 2>/dev/null has_gpu = len(gpu_info) > 0 print("GPU detected?" , has_gpu) if has_gpu: # Common working wheel for Colab: cu118 (CUDA 11.8) — fallback to CPU if this fails. try: subprocess.run([pip_cmd, 'install', 'torch', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu118'], check=True) except Exception as e: print("CUDA wheel install failed, falling back to CPU torch:", e) subprocess.run([pip_cmd, 'install', 'torch', '--index-url', 'https://download.pytorch.org/whl/cpu'], check=True) else: subprocess.run([pip_cmd, 'install', 'torch', '--index-url', 'https://download.pytorch.org/whl/cpu'], check=True)

# 7) Install other Python requirements for ComfyUI (use requirements.txt if present)
req_file = os.path.join(REPO_DIR, 'requirements.txt') if os.path.exists(req_file): # Use pip from our venv to install requirements; some items (like torch) are already installed and will be skipped subprocess.run([pip_cmd, 'install', '-r', req_file], check=False) # allow non-zero exit if optional packages fail else: print("No requirements.txt found in repo — continuing.")

# 8) Install/upgade common ComfyUI node repos in the custom_nodes folder (attempts to update existing ones)
# Add any node repos you use here. Example list — adjust or add your own repos into Drive/custom_nodes as needed.
example_nodes = { # 'ComfyUI-ExampleNode': 'https://github.com/example/ComfyUI-ExampleNode.git' # If you have nodes you want pre-cloned into Drive, skip cloning here. }

for name, repo in example_nodes.items(): target = os.path.join(NODES_DRIVE, name) if not os.path.exists(target): subprocess.run(['git', 'clone', '--depth', '1', repo, target], check=False) else: subprocess.run(['git', '-C', target, 'pull'], check=False)

# If you keep custom node repos in Drive manually, they will be available via symlink above.
# 9) Optional: read a saved COMFYUI password or ngrok token from Drive
COMFY_PASS_FILE = os.path.join(DRIVE_ROOT, 'comfy_password.txt') comfy_pass = None if os.path.exists(COMFY_PASS_FILE): with open(COMFY_PASS_FILE, 'r') as f: comfy_pass = f.read().strip() print("Loaded ComfyUI password from Drive.") if comfy_pass: os.environ['COMFYUI_PASSWORD'] = comfy_pass

ngrok_token = None if os.path.exists(NGROK_TOKEN_FILE): with open(NGROK_TOKEN_FILE, 'r') as f: ngrok_token = f.read().strip() print("Loaded ngrok token from Drive (will use it to expose the UI).")

# 10) Optional: install pyngrok if we will expose the UI
expose_with_ngrok = bool(ngrok_token) if expose_with_ngrok: subprocess.run([pip_cmd, 'install', 'pyngrok'], check=True) from pyngrok import ngrok if ngrok_token: ngrok.set_auth_token(ngrok_token)

# 11) Final: run ComfyUI (in background) and optionally create an ngrok tunnel to the port
COMFY_PORT = 8188 start_cmd = [bin_py, os.path.join(REPO_DIR, 'main.py'), '--host', '0.0.0.0', '--port', str(COMFY_PORT)]

# Launch ComfyUI
print("Starting ComfyUI (this will run in background)...") proc = subprocess.Popen(start_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Wait a bit for server to start, show logs
time.sleep(3)

# print a few lines of stdout if available
try: out = proc.stdout.read(2000) if out: print(out) except Exception: pass

if expose_with_ngrok: # create http tunnel to the ComfyUI port public_url = ngrok.connect(COMFY_PORT, "http") print("ComfyUI should be available at:", public_url) else: print("ComfyUI started on the notebook VM. To access it from your browser you can:") print(" - Use an SSH tunnel (advanced), or") print(" - Provide an ngrok token (put token in Drive at:", NGROK_TOKEN_FILE, ") and re-run this cell.") print("Local notebook URL (inside runtime) : http://127.0.0.1:%d" % COMFY_PORT)

print("\nNotes:") print(" - Models and custom nodes in Drive are linked into the runtime at /content/ComfyUI/models and /content/ComfyUI/custom_nodes.") print(" - To update ComfyUI and nodes later, re-run this cell; it pulls the latest changes from the Git repos.") print(" - Stop the runtime when finished to avoid idle GPU charges (Runtime → Manage sessions → terminate).")

# Optionally stream logs to cell output for debugging:
# If you want continuous logs uncomment the following block (may block the cell until kernel stop):
for line in proc.stdout:
print(line, end='')
==========================================================================================